# **Cyclistic bike sharing case-study**  


<img src='/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Case_Study_Bike_Sharing/python_script/cyclistic.png' width=”5” height=”5”>



### **Data Cleaning and Modelling Notebook**

 **Author**: Karthik Bhaktha  
 **Created**: April 24th, 2022  
 **Data Source**: [Divvy](https://divvy-tripdata.s3.amazonaws.com/index.html)  
 **Data by**: Motivate International Inc. [License](https://www.divvybikes.com/data-license-agreement)  

Purpose of this notebook is to create a master dataset that can be used to perform an analysis to answer the question:  
**How do annual members and casual riders use Cyclistic bikes differently?**  
This will help understand how we can employ strategy leading towards converstion of **casual** members into **annual** members.  
According to the financial analyst doing so could lead to earning higher profits.


The data source is a collection of 12 CSV files.  
Each file contains one month of data covering 12 months from Jan 2021 to December 2021.  

### Cleaning steps performed in this notebook:
- Merge all CSV files into a single dataframe.
- Create calculated columns: total_seconds and total_distance.
- Check for data integrity.
- Deal with null values.

In [1]:
import pandas as pd
import missingno as msno #for visualizing null values
import os
import haversine as hs
from haversine import Unit #for calculating distance using coordinates
import datetime as dt
import numpy as np
import plotly.express as px

Creating a function that reads the file path.    
This function will return a list of 13 filenames.  
Using the list to read and concatenate creating a master dataframe. 

In [2]:
'''
    For the given path, get the List of all files in the directory tree 
'''
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

The code below:  
- Reads the file path
- Concatenates the dataframe into a master dataframe.

In [3]:
#Creating the master dataframe
path =  getListOfFiles('/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/data_set')
# run the code below to confirm the path list.
print(path)
master_df = pd.concat(map(pd.read_csv,path), ignore_index= True)

['/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/data_set/202109-divvy-tripdata.csv', '/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/data_set/202104-divvy-tripdata.csv', '/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/data_set/202107-divvy-tripdata.csv', '/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/data_set/202111-divvy-tripdata.csv', '/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/data_set/202101-divvy-tripdata.csv', '/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/data_set/202102-divvy-tripdata.csv', '/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/data_set/202112-divvy-tripdata.csv', '/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/data_set/202103-divvy-tripdata.csv', '/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/data_set/202110-divvy-tripda

In [4]:
#checking the shape of master_df
master_df.info()
master_df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5595063 entries, 0 to 5595062
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 554.9+ MB


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,9DC7B962304CBFD8,electric_bike,2021-09-28 16:07:10,2021-09-28 16:09:54,NaN,NaN,NaN,NaN,41.890000,-87.680000,41.89000,-87.670000,casual
1,F930E2C6872D6B32,electric_bike,2021-09-28 14:24:51,2021-09-28 14:40:05,NaN,NaN,NaN,NaN,41.940000,-87.640000,41.98000,-87.670000,casual
2,6EF72137900BB910,electric_bike,2021-09-28 00:20:16,2021-09-28 00:23:57,NaN,NaN,NaN,NaN,41.810000,-87.720000,41.80000,-87.720000,casual
3,78D1DE133B3DBF55,electric_bike,2021-09-28 14:51:17,2021-09-28 15:00:06,NaN,NaN,NaN,NaN,41.800000,-87.720000,41.81000,-87.720000,casual
4,E03D4ACDCAEF6E00,electric_bike,2021-09-28 09:53:12,2021-09-28 10:03:44,NaN,NaN,NaN,NaN,41.880000,-87.740000,41.88000,-87.710000,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595058,EF56D7D1D612AC11,electric_bike,2021-05-20 16:32:14,2021-05-20 16:35:39,Blackstone Ave & Hyde Park Blvd,13398,NaN,NaN,41.802581,-87.590230,41.80000,-87.600000,member
5595059,745191CB9F21DE3C,classic_bike,2021-05-29 16:40:37,2021-05-29 17:22:37,Sheridan Rd & Montrose Ave,TA1307000107,Michigan Ave & Oak St,13042,41.961670,-87.654640,41.90096,-87.623777,casual
5595060,428575BAA5356BFF,electric_bike,2021-05-31 14:24:54,2021-05-31 14:31:38,Sheridan Rd & Montrose Ave,TA1307000107,NaN,NaN,41.961525,-87.654651,41.95000,-87.650000,member
5595061,FC8A4A7AB7249662,electric_bike,2021-05-25 16:01:33,2021-05-25 16:07:37,Sheridan Rd & Montrose Ave,TA1307000107,NaN,NaN,41.961654,-87.654721,41.98000,-87.660000,member


The Code below:
- Loops throough each dataframe
- finds the size of each csv file 
- calculates the total size

In [5]:
#checking if the merging csv was successful by comparing the size
total = 0
for i in range(len(path)):
    df = pd.read_csv(path[i])
    '''uncheck the comment below to confirm the size of each dtaframe'''
    #print("Size = ", df.size, "of path", path[i])
    total += df.size


print("The size of merged files and total size from loop above are equal",total, master_df.size)

The size of merged files and total size from loop above are equal 72735819 72735819


### Creating a new datafram called final to perform further operations without disturbing the master dataframe

In [6]:
#creating a new df to store the datetime changes.
final_df = master_df

Checking for null rows in the final_df.

In [7]:
final_df.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    690809
start_station_id      690806
end_station_name      739170
end_station_id        739170
start_lat                  0
start_lng                  0
end_lat                 4771
end_lng                 4771
member_casual              0
dtype: int64

Since there are quite a few missing values in the columns start_station_name and end_station name, almost 700,000 values are missing, we will perform analysis on the subset of the dataset containing those columns.   
Store the subset in a csv file called station_name_merged

In [8]:
# creating a subet dataset with station_names with all the null values removed.``
station_merged = final_df.dropna(axis=0)
station_merged.isna().sum()


ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

We will store this dataframe as a csv file and comeback to it later for further analysis.

In [9]:
station_merged.to_csv('/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/station_merged.csv')
size = station_merged.size
master_size = master_df.size 
print("This csv file is of size: ", size, "Whereas the master_df is of size: ", master_size)

This csv file is of size:  59647926 Whereas the master_df is of size:  72735819


Moving on, we will remove the columns and rows with missing data from final_df.


Dropping columns with station name and station ID, since there are lots of missing values.
Drop rows with null values from end_lat and end_lng. Dropping these rows should not affect the analysis as much.

In [10]:
# removing the columns that wont be used in this analysis
final_df.drop(columns=['start_station_name','start_station_id','end_station_name','end_station_id'], axis = 1, inplace = True)
final_df.info()
final_df
final_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5595063 entries, 0 to 5595062
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ride_id        object 
 1   rideable_type  object 
 2   started_at     object 
 3   ended_at       object 
 4   start_lat      float64
 5   start_lng      float64
 6   end_lat        float64
 7   end_lng        float64
 8   member_casual  object 
dtypes: float64(4), object(5)
memory usage: 384.2+ MB


ride_id             0
rideable_type       0
started_at          0
ended_at            0
start_lat           0
start_lng           0
end_lat          4771
end_lng          4771
member_casual       0
dtype: int64

In [11]:
# removing the rows with null values
final_df.dropna(subset=['end_lat', 'end_lng'], inplace=True)
final_df.info()
final_df
# checking for null values again after removing the columns and rows
final_df.isna().sum()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5590292 entries, 0 to 5595062
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ride_id        object 
 1   rideable_type  object 
 2   started_at     object 
 3   ended_at       object 
 4   start_lat      float64
 5   start_lng      float64
 6   end_lat        float64
 7   end_lng        float64
 8   member_casual  object 
dtypes: float64(4), object(5)
memory usage: 426.5+ MB


ride_id          0
rideable_type    0
started_at       0
ended_at         0
start_lat        0
start_lng        0
end_lat          0
end_lng          0
member_casual    0
dtype: int64

Converting the columns 'started_at', 'ended_at' from data type object to datetime.  
This is to calculate the duration of the ride.

In [12]:
#typecasting 'started_at', 'ended_at' into datetime using .astype('datetime64[ns]) method.
final_df.loc[:,['started_at', 'ended_at']] = final_df.loc[:,['started_at', 'ended_at']].astype('datetime64[ns]')

#Creating a calculated column total_time = ended_at - started_at
final_df['total_time'] = abs(final_df.ended_at - final_df.started_at)

#confirming if the data type has changed
final_df.info()
#confirming if the type casting has not altered the columns
final_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5590292 entries, 0 to 5595062
Data columns (total 10 columns):
 #   Column         Dtype          
---  ------         -----          
 0   ride_id        object         
 1   rideable_type  object         
 2   started_at     datetime64[ns] 
 3   ended_at       datetime64[ns] 
 4   start_lat      float64        
 5   start_lng      float64        
 6   end_lat        float64        
 7   end_lng        float64        
 8   member_casual  object         
 9   total_time     timedelta64[ns]
dtypes: datetime64[ns](2), float64(4), object(3), timedelta64[ns](1)
memory usage: 469.2+ MB


,ride_id,rideable_type,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,member_casual,total_time
0,9DC7B962304CBFD8,electric_bike,2021-09-28 16:07:10,2021-09-28 16:09:54,41.890000,-87.680000,41.89000,-87.670000,casual,0 days 00:02:44
1,F930E2C6872D6B32,electric_bike,2021-09-28 14:24:51,2021-09-28 14:40:05,41.940000,-87.640000,41.98000,-87.670000,casual,0 days 00:15:14
2,6EF72137900BB910,electric_bike,2021-09-28 00:20:16,2021-09-28 00:23:57,41.810000,-87.720000,41.80000,-87.720000,casual,0 days 00:03:41
3,78D1DE133B3DBF55,electric_bike,2021-09-28 14:51:17,2021-09-28 15:00:06,41.800000,-87.720000,41.81000,-87.720000,casual,0 days 00:08:49
4,E03D4ACDCAEF6E00,electric_bike,2021-09-28 09:53:12,2021-09-28 10:03:44,41.880000,-87.740000,41.88000,-87.710000,casual,0 days 00:10:32
...,...,...,...,...,...,...,...,...,...,...
5595058,EF56D7D1D612AC11,electric_bike,2021-05-20 16:32:14,2021-05-20 16:35:39,41.802581,-87.590230,41.80000,-87.600000,member,0 days 00:03:25
5595059,745191CB9F21DE3C,classic_bike,2021-05-29 16:40:37,2021-05-29 17:22:37,41.961670,-87.654640,41.90096,-87.623777,casual,0 days 00:42:00
5595060,428575BAA5356BFF,electric_bike,2021-05-31 14:24:54,2021-05-31 14:31:38,41.961525,-87.654651,41.95000,-87.650000,member,0 days 00:06:44
5595061,FC8A4A7AB7249662,electric_bike,2021-05-25 16:01:33,2021-05-25 16:07:37,41.961654,-87.654721,41.98000,-87.660000,member,0 days 00:06:04


As you can see the calculated column has resulted in a format that includes number of days and hh:mm:ss format.  
Extracting only seconds from the column and add it to a new column called total_seconds. 

In [13]:
final_df.isna().sum()

ride_id          0
rideable_type    0
started_at       0
ended_at         0
start_lat        0
start_lng        0
end_lat          0
end_lng          0
member_casual    0
total_time       0
dtype: int64

In [14]:
'''This loop used iterows and it too about 2m:30s to run
Using dictinary iteration drops the speed to iterate significantly to only 20secs.
'''
#calculating the duration of the ride in seconds and adding it to a new column.
# for index, rows in final_df.iterrows():
#     final_df.at[index, 'total_seconds'] = rows['total_time'].total_seconds()

'This loop used iterows and it too about 2m:30s to run\nUsing dictinary iteration drops the speed to iterate significantly to only 20secs.\n'

Extracting just the total_time column to make the loop run faster, 
this is because the loop does not read through all the columns in the dataframe.

In [15]:
# reset_index
final_df.reset_index(drop=True, inplace=True)

In [16]:
# extracting total_time column from final_df
total_time_df = final_df[['total_time']]
# total_time_df.info()
total_time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5590292 entries, 0 to 5590291
Data columns (total 1 columns):
 #   Column      Dtype          
---  ------      -----          
 0   total_time  timedelta64[ns]
dtypes: timedelta64[ns](1)
memory usage: 42.7 MB


In [17]:
'''Improved loop using dictionary iteration:'''
time_list_seconds = []
time_list_minutes = []
count = 0
#calculating the duration of the ride in seconds and adding it to a new column.
for rows in total_time_df.to_dict('records'):
    time_list_seconds.append(rows['total_time'].total_seconds())
    time_list_minutes.append((rows['total_time'].total_seconds())/60)
    count += 1
    
print(count)
total_seconds_dict = {'duration_seconds':time_list_seconds}
total_minutes_dict = {'duration_minutes':time_list_minutes}


5590292


Total_time_dict is now a dictionary.  
in the code cell below we will convert the dictinary into a datafrane and append it to the final_df.

In [18]:
# converting dict to df
total_seconds_df = pd.DataFrame.from_dict(total_seconds_dict)
total_minutes_df = pd.DataFrame.from_dict(total_minutes_dict)

total_seconds_df.reset_index(drop=True, inplace=True)
total_minutes_df.reset_index(drop=True, inplace=True)

final_df = pd.concat([final_df, total_seconds_df, total_minutes_df],axis=1)

print(final_df.shape[0])
print(final_df.shape[1])
final_df.isna().sum()

5590292
12


ride_id             0
rideable_type       0
started_at          0
ended_at            0
start_lat           0
start_lng           0
end_lat             0
end_lng             0
member_casual       0
total_time          0
duration_seconds    0
duration_minutes    0
dtype: int64

In [19]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5590292 entries, 0 to 5590291
Data columns (total 12 columns):
 #   Column            Dtype          
---  ------            -----          
 0   ride_id           object         
 1   rideable_type     object         
 2   started_at        datetime64[ns] 
 3   ended_at          datetime64[ns] 
 4   start_lat         float64        
 5   start_lng         float64        
 6   end_lat           float64        
 7   end_lng           float64        
 8   member_casual     object         
 9   total_time        timedelta64[ns]
 10  duration_seconds  float64        
 11  duration_minutes  float64        
dtypes: datetime64[ns](2), float64(6), object(3), timedelta64[ns](1)
memory usage: 511.8+ MB


In [20]:
# dropping total_time
final_df.drop(columns='total_time', axis = 1, inplace = True)
final_df.loc[:,['duration_seconds', 'duration_minutes']] = final_df.loc[:,['duration_seconds', 'duration_minutes']].astype('int64')
final_df.info()
final_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5590292 entries, 0 to 5590291
Data columns (total 11 columns):
 #   Column            Dtype         
---  ------            -----         
 0   ride_id           object        
 1   rideable_type     object        
 2   started_at        datetime64[ns]
 3   ended_at          datetime64[ns]
 4   start_lat         float64       
 5   start_lng         float64       
 6   end_lat           float64       
 7   end_lng           float64       
 8   member_casual     object        
 9   duration_seconds  int64         
 10  duration_minutes  int64         
dtypes: datetime64[ns](2), float64(4), int64(2), object(3)
memory usage: 469.2+ MB


,ride_id,rideable_type,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,member_casual,duration_seconds,duration_minutes
0,9DC7B962304CBFD8,electric_bike,2021-09-28 16:07:10,2021-09-28 16:09:54,41.89,-87.68,41.89,-87.67,casual,164,2
1,F930E2C6872D6B32,electric_bike,2021-09-28 14:24:51,2021-09-28 14:40:05,41.94,-87.64,41.98,-87.67,casual,914,15
2,6EF72137900BB910,electric_bike,2021-09-28 00:20:16,2021-09-28 00:23:57,41.81,-87.72,41.80,-87.72,casual,221,3
3,78D1DE133B3DBF55,electric_bike,2021-09-28 14:51:17,2021-09-28 15:00:06,41.80,-87.72,41.81,-87.72,casual,529,8
4,E03D4ACDCAEF6E00,electric_bike,2021-09-28 09:53:12,2021-09-28 10:03:44,41.88,-87.74,41.88,-87.71,casual,632,10


Creating a new column with total distance travelled.  
Formula for converting coordinates into distance using [haversine function](https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b)  
Using haversine library to convert the coordinates into distance.  
By default haversine returns the distance in kms.

In [21]:
'''Improved loop in the code cells below'''
#iterating over dataframe to calulate the haversine function

# for index, row in final_df.iterrows():
    # calculating distance in miles
    # loc1= (row['start_lat'], row['start_lng'] )
    # loc2= (row['end_lat'], row['end_lng'] )
    # final_df.at[index, 'total_distance_miles'] = hs.haversine(loc1, loc2, unit= Unit.MILES )
     


'Improved loop in the code cells below'

In [22]:
#extracting just the lat and lng columns for faster iteration
lat_lng_df = final_df[['start_lat','start_lng','end_lat','end_lng']]
lat_lng_df.head()

,start_lat,start_lng,end_lat,end_lng
0,41.89,-87.68,41.89,-87.67
1,41.94,-87.64,41.98,-87.67
2,41.81,-87.72,41.80,-87.72
3,41.80,-87.72,41.81,-87.72
4,41.88,-87.74,41.88,-87.71


In [23]:
'''Using dictionary iteration'''
distance_list = []

for row in lat_lng_df.to_dict('records'):
    distance_list.append(hs.haversine((row['start_lat'], row['start_lng'] ), (row['end_lat'], row['end_lng'] ), unit= Unit.MILES ))

distance_dictionary = {"distance_miles": distance_list}       

In [24]:
distance_df = pd.DataFrame.from_dict(distance_dictionary)

distance_df.reset_index(drop=True, inplace=True)
final_df.reset_index(drop=True, inplace=True)

final_df = pd.concat([final_df, distance_df], axis=1)

final_df.info()
final_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5590292 entries, 0 to 5590291
Data columns (total 12 columns):
 #   Column            Dtype         
---  ------            -----         
 0   ride_id           object        
 1   rideable_type     object        
 2   started_at        datetime64[ns]
 3   ended_at          datetime64[ns]
 4   start_lat         float64       
 5   start_lng         float64       
 6   end_lat           float64       
 7   end_lng           float64       
 8   member_casual     object        
 9   duration_seconds  int64         
 10  duration_minutes  int64         
 11  distance_miles    float64       
dtypes: datetime64[ns](2), float64(5), int64(2), object(3)
memory usage: 511.8+ MB


,ride_id,rideable_type,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,member_casual,duration_seconds,duration_minutes,distance_miles
0,9DC7B962304CBFD8,electric_bike,2021-09-28 16:07:10,2021-09-28 16:09:54,41.89,-87.68,41.89,-87.67,casual,164,2,0.514351
1,F930E2C6872D6B32,electric_bike,2021-09-28 14:24:51,2021-09-28 14:40:05,41.94,-87.64,41.98,-87.67,casual,914,15,3.164496
2,6EF72137900BB910,electric_bike,2021-09-28 00:20:16,2021-09-28 00:23:57,41.81,-87.72,41.80,-87.72,casual,221,3,0.690934
3,78D1DE133B3DBF55,electric_bike,2021-09-28 14:51:17,2021-09-28 15:00:06,41.80,-87.72,41.81,-87.72,casual,529,8,0.690934
4,E03D4ACDCAEF6E00,electric_bike,2021-09-28 09:53:12,2021-09-28 10:03:44,41.88,-87.74,41.88,-87.71,casual,632,10,1.543294


In [25]:
# Using Ceil function 
final_df['distance_miles'] = final_df['distance_miles'].apply(np.ceil)
# final_df[:,['distance_miles']] = final_df[:,['distance_miles']].astype('int64')
final_df

,ride_id,rideable_type,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,member_casual,duration_seconds,duration_minutes,distance_miles
0,9DC7B962304CBFD8,electric_bike,2021-09-28 16:07:10,2021-09-28 16:09:54,41.890000,-87.680000,41.89000,-87.670000,casual,164,2,1.0
1,F930E2C6872D6B32,electric_bike,2021-09-28 14:24:51,2021-09-28 14:40:05,41.940000,-87.640000,41.98000,-87.670000,casual,914,15,4.0
2,6EF72137900BB910,electric_bike,2021-09-28 00:20:16,2021-09-28 00:23:57,41.810000,-87.720000,41.80000,-87.720000,casual,221,3,1.0
3,78D1DE133B3DBF55,electric_bike,2021-09-28 14:51:17,2021-09-28 15:00:06,41.800000,-87.720000,41.81000,-87.720000,casual,529,8,1.0
4,E03D4ACDCAEF6E00,electric_bike,2021-09-28 09:53:12,2021-09-28 10:03:44,41.880000,-87.740000,41.88000,-87.710000,casual,632,10,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5590287,EF56D7D1D612AC11,electric_bike,2021-05-20 16:32:14,2021-05-20 16:35:39,41.802581,-87.590230,41.80000,-87.600000,member,205,3,1.0
5590288,745191CB9F21DE3C,classic_bike,2021-05-29 16:40:37,2021-05-29 17:22:37,41.961670,-87.654640,41.90096,-87.623777,casual,2520,42,5.0
5590289,428575BAA5356BFF,electric_bike,2021-05-31 14:24:54,2021-05-31 14:31:38,41.961525,-87.654651,41.95000,-87.650000,member,404,6,1.0
5590290,FC8A4A7AB7249662,electric_bike,2021-05-25 16:01:33,2021-05-25 16:07:37,41.961654,-87.654721,41.98000,-87.660000,member,364,6,2.0


In [26]:
final_df['distance_miles'] = final_df['distance_miles'].astype("int64")
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5590292 entries, 0 to 5590291
Data columns (total 12 columns):
 #   Column            Dtype         
---  ------            -----         
 0   ride_id           object        
 1   rideable_type     object        
 2   started_at        datetime64[ns]
 3   ended_at          datetime64[ns]
 4   start_lat         float64       
 5   start_lng         float64       
 6   end_lat           float64       
 7   end_lng           float64       
 8   member_casual     object        
 9   duration_seconds  int64         
 10  duration_minutes  int64         
 11  distance_miles    int64         
dtypes: datetime64[ns](2), float64(4), int64(3), object(3)
memory usage: 511.8+ MB


Distance_miles is the direct distance between the coordinates.  
The distance may slightly vary on the map because it measures the distance betwween two coordinates. 
This means that the distance may not be accurate.

Checking the unique values in rideable_type.

In [27]:
final_df.sort_values(by = ['started_at'], ascending = True, inplace= True)
final_df.info()
final_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5590292 entries, 2347568 to 2520848
Data columns (total 12 columns):
 #   Column            Dtype         
---  ------            -----         
 0   ride_id           object        
 1   rideable_type     object        
 2   started_at        datetime64[ns]
 3   ended_at          datetime64[ns]
 4   start_lat         float64       
 5   start_lng         float64       
 6   end_lat           float64       
 7   end_lng           float64       
 8   member_casual     object        
 9   duration_seconds  int64         
 10  duration_minutes  int64         
 11  distance_miles    int64         
dtypes: datetime64[ns](2), float64(4), int64(3), object(3)
memory usage: 554.5+ MB


,ride_id,rideable_type,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,member_casual,duration_seconds,duration_minutes,distance_miles
2347568,A3F8D895163BBB49,electric_bike,2021-01-01 00:02:05,2021-01-01 00:12:39,41.980000,-87.650000,41.980000,-87.660000,member,634,10,1
2352232,0D139A3203274B87,classic_bike,2021-01-01 00:02:24,2021-01-01 00:08:39,41.834734,-87.625813,41.842052,-87.617000,member,375,6,1
2313882,C7AE8E9CDB197A8E,classic_bike,2021-01-01 00:06:55,2021-01-01 00:26:36,41.925858,-87.638973,41.906866,-87.626217,member,1181,19,2
2307993,2633EB2B8A99F5CB,electric_bike,2021-01-01 00:12:13,2021-01-01 00:20:06,41.929528,-87.707899,41.920000,-87.720000,member,473,7,1
2307864,3097EF26414C7016,classic_bike,2021-01-01 00:12:21,2021-01-01 00:12:33,41.963982,-87.638181,41.963982,-87.638181,member,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2551796,7B3D0E8AB0842D5E,classic_bike,2021-12-31 23:58:21,2022-01-01 00:46:56,41.886024,-87.624117,41.875885,-87.640795,casual,2915,48,2
2551797,B5AEBEF3B5F41C77,electric_bike,2021-12-31 23:58:45,2022-01-01 00:47:07,41.885992,-87.624782,41.875885,-87.640795,casual,2902,48,2
2467701,CE1BE016BCE85CCB,electric_bike,2021-12-31 23:59:27,2022-01-01 00:32:34,41.875831,-87.631348,41.875933,-87.630585,casual,1987,33,1
2472399,B0466FF51982DE4B,electric_bike,2021-12-31 23:59:39,2022-01-01 00:21:08,41.881130,-87.624046,41.864059,-87.623727,member,1289,21,2


In [28]:
#resetting index
final_df.reset_index(drop=True, inplace=True)
final_df

,ride_id,rideable_type,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,member_casual,duration_seconds,duration_minutes,distance_miles
0,A3F8D895163BBB49,electric_bike,2021-01-01 00:02:05,2021-01-01 00:12:39,41.980000,-87.650000,41.980000,-87.660000,member,634,10,1
1,0D139A3203274B87,classic_bike,2021-01-01 00:02:24,2021-01-01 00:08:39,41.834734,-87.625813,41.842052,-87.617000,member,375,6,1
2,C7AE8E9CDB197A8E,classic_bike,2021-01-01 00:06:55,2021-01-01 00:26:36,41.925858,-87.638973,41.906866,-87.626217,member,1181,19,2
3,2633EB2B8A99F5CB,electric_bike,2021-01-01 00:12:13,2021-01-01 00:20:06,41.929528,-87.707899,41.920000,-87.720000,member,473,7,1
4,3097EF26414C7016,classic_bike,2021-01-01 00:12:21,2021-01-01 00:12:33,41.963982,-87.638181,41.963982,-87.638181,member,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5590287,7B3D0E8AB0842D5E,classic_bike,2021-12-31 23:58:21,2022-01-01 00:46:56,41.886024,-87.624117,41.875885,-87.640795,casual,2915,48,2
5590288,B5AEBEF3B5F41C77,electric_bike,2021-12-31 23:58:45,2022-01-01 00:47:07,41.885992,-87.624782,41.875885,-87.640795,casual,2902,48,2
5590289,CE1BE016BCE85CCB,electric_bike,2021-12-31 23:59:27,2022-01-01 00:32:34,41.875831,-87.631348,41.875933,-87.630585,casual,1987,33,1
5590290,B0466FF51982DE4B,electric_bike,2021-12-31 23:59:39,2022-01-01 00:21:08,41.881130,-87.624046,41.864059,-87.623727,member,1289,21,2


We will be checking for data integrity.
- Check min and max date
- Check min and max duration_seconds and duration_minutes
- Check for negative durations (started_at > ended_at)
- Check for negative distance
- Check unique rideable_type (renaming to bike_type)

In [29]:
# checkig min and max date in the final_df to see if there are any errors in the date.
print(final_df.started_at.min())
print(final_df.ended_at.min())
print(final_df.started_at.max())
print(final_df.ended_at.max())

2021-01-01 00:02:05
2021-01-01 00:08:39
2021-12-31 23:59:48
2022-01-03 17:32:18


For this analysis we are interested in using started_at column as mean to tell when the rider start inititated the ride, we can ignore the ended_at

In [33]:
# Check if started_at > ended_at because ride cannot start after the end time and vice versa
start_grtr_end = final_df[final_df['started_at'] > final_df['ended_at']] 
start_grtr_end

'''There are 146 rows in the dataset that has a start time greater than end time
We will filter the data out and store the filtered data into final_df'''

#filtering out rows and storing it in final_df such that only the rows where start time is less than end time.
final_df = final_df[final_df['started_at'] < final_df['ended_at']] 
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5589640 entries, 0 to 5590291
Data columns (total 12 columns):
 #   Column            Dtype         
---  ------            -----         
 0   ride_id           object        
 1   rideable_type     object        
 2   started_at        datetime64[ns]
 3   ended_at          datetime64[ns]
 4   start_lat         float64       
 5   start_lng         float64       
 6   end_lat           float64       
 7   end_lng           float64       
 8   member_casual     object        
 9   duration_seconds  int64         
 10  duration_minutes  int64         
 11  distance_miles    int64         
dtypes: datetime64[ns](2), float64(4), int64(3), object(3)
memory usage: 554.4+ MB


In [34]:
# checking for duration min and max
print(final_df.duration_seconds.min())
print(final_df.duration_minutes.min())
print(final_df.duration_seconds.max())
print(final_df.duration_minutes.max())

'''Exploring more about why the data has such a huge number for duration'''

1
0
3356649
55944


In [40]:
'''There are a lots of rows that have unreasonably long duration of rides.
Check how many rows are of duration that are over 24 hours or 1440 mins'''

sort_duration_df = final_df.sort_values(by = ['duration_minutes'], ascending = False)
sort_duration_df.head(100)


,ride_id,rideable_type,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,member_casual,duration_seconds,duration_minutes,distance_miles
1333674,F043F0F6A1AA4F85,docked_bike,2021-06-05 02:27:26,2021-07-13 22:51:35,41.886024,-87.624117,41.877618,-87.673895,casual,3356649,55944,3
1329643,7F0578ABF030FC83,docked_bike,2021-06-04 22:03:33,2021-07-13 14:15:14,41.892278,-87.612043,41.889955,-87.680651,casual,3341501,55691,4
737218,BDA1217EC8532C7B,docked_bike,2021-05-02 02:56:07,2021-06-08 13:37:43,41.877181,-87.627844,41.872596,-87.633502,casual,3235296,53921,1
1367882,E6E1E5BFFD72ECEC,docked_bike,2021-06-05 23:33:51,2021-07-12 13:55:14,41.886021,-87.630876,41.889955,-87.680651,casual,3162083,52701,3
2180877,99415B003B7E7EEE,docked_bike,2021-07-08 19:29:49,2021-08-11 21:56:58,41.929567,-87.707857,41.889955,-87.680651,casual,2946429,49107,4
...,...,...,...,...,...,...,...,...,...,...,...,...
1655151,152B6C7AC3DACCEA,docked_bike,2021-06-16 19:29:31,2021-07-01 18:02:06,41.856268,-87.613348,41.889955,-87.680651,casual,1290755,21512,5
1390671,3FD1BB5F9FD42C81,docked_bike,2021-06-06 18:08:39,2021-06-21 12:26:08,41.881032,-87.624084,41.799568,-87.594747,casual,1275449,21257,6
3829954,8BAA2D80763C5529,docked_bike,2021-09-10 02:18:47,2021-09-24 20:28:22,41.929546,-87.643118,41.931248,-87.644336,casual,1274975,21249,1
1759066,A3AA4F264AB0CD28,docked_bike,2021-06-20 11:47:07,2021-07-05 03:17:16,41.884114,-87.654264,41.935337,-87.716889,casual,1265409,21090,5


In [41]:
#checking rides that are longer tham 24hours or 1440mins
sorted_24hr = final_df[final_df['duration_minutes'] > 1440]
sorted_24hr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1331 entries, 199 to 5590087
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ride_id           1331 non-null   object        
 1   rideable_type     1331 non-null   object        
 2   started_at        1331 non-null   datetime64[ns]
 3   ended_at          1331 non-null   datetime64[ns]
 4   start_lat         1331 non-null   float64       
 5   start_lng         1331 non-null   float64       
 6   end_lat           1331 non-null   float64       
 7   end_lng           1331 non-null   float64       
 8   member_casual     1331 non-null   object        
 9   duration_seconds  1331 non-null   int64         
 10  duration_minutes  1331 non-null   int64         
 11  distance_miles    1331 non-null   int64         
dtypes: datetime64[ns](2), float64(4), int64(3), object(3)
memory usage: 135.2+ KB


From the output above we can see that there are about 1331 riders who used the bikes for over 24hours, which is about ~0.025% of population.
Therefore we can filter out this row, so as to avoid altering our analysis.


In [43]:
#filtering out 1331 rows from the final_df
final_df = final_df[final_df['duration_minutes'] < 1331]
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5588023 entries, 0 to 5590291
Data columns (total 12 columns):
 #   Column            Dtype         
---  ------            -----         
 0   ride_id           object        
 1   rideable_type     object        
 2   started_at        datetime64[ns]
 3   ended_at          datetime64[ns]
 4   start_lat         float64       
 5   start_lng         float64       
 6   end_lat           float64       
 7   end_lng           float64       
 8   member_casual     object        
 9   duration_seconds  int64         
 10  duration_minutes  int64         
 11  distance_miles    int64         
dtypes: datetime64[ns](2), float64(4), int64(3), object(3)
memory usage: 554.2+ MB


In [44]:
# rechecking the condition
print(final_df.duration_seconds.min())
print(final_df.duration_minutes.min())
print(final_df.duration_seconds.max())
print(final_df.duration_minutes.max())

1
0
79821
1330


In [ ]:
# renaming rideable_type to bike_type
final_df.rename(columns = {'rideable_type':'bike_type'}, inplace = True)
final_df.info()

In [ ]:
unique_rides = final_df['bike_type'].unique()
print(unique_rides)
unique_member = final_df['member_casual'].unique()
print(unique_member)

In [ ]:
# checking total_seconds column for any unusual values.
print(final_df.total_seconds.min())
print(final_df.total_seconds.max())
'''Fixed this in the code cell where time difference is calculated'''
# filter out columns with negetive total_seconds
# less_than_zero_df = final_df[final_df['total_seconds'] < 0]
# less_than_zero_df.info()
# less_than_zero_df

In [ ]:
# checking if start time is larger than end time, because you cannot travel back in time.
result_df = final_df[final_df['started_at'] > final_df['ended_at']]
result_df.info()
'''There are 147 rows where started_at is greater than ended_at. It is not possible to travel back in time... yet.'''

In [ ]:
# removing the rows where started_at is greater than ended_at
final_df=  final_df[final_df['started_at'] < final_df['ended_at']]
final_df.info()

In [ ]:
# checking the distance for any invalid values like negetive distance
print(final_df.distance_miles.min())
print(final_df.distance_miles.max())
result_df = final_df[final_df['distance_miles'] == 739.1357259057336]
result_df
# checking for the largest distance travelled by a rider
desc_dist_df = final_df.sort_values(by = ['distance_miles'], ascending = False)
desc_dist_df

'''Ride lasting only 155 seconds but distance travelled is 739miles, that is not possible and it is possibly an error'''


In [ ]:
# checking the number of rows in the final_df with distance = 0 miles
'''Zero miles does not necessarily mean that it is a error, it could be that the rider ended the ride from where it started, meaning 
the ride ended at the same location where it began'''
distance_zero = final_df[final_df['distance_miles'] == 0]
distance_zero


In [ ]:
# sorting the dataframe in desc order of total_seconds
desc_seconds = final_df.sort_values(by = ['total_seconds'], ascending = False)
desc_seconds

Aadding a column with days of the week as 0 for Monday to 6 for Sunday.  
Also add day name column. Monday to Sunday

In [ ]:
# adding days of week names
final_df['days_of_week'] = final_df['started_at'].dt.day_name()
# final_df.head()
print('Unique days',final_df['days_of_week'].unique())

# adding days of week values 0-6
final_df['days_of_week_val'] = final_df['started_at'].dt.dayofweek
# final_df.head()
print('Unique days value' ,final_df['days_of_week_val'].unique())

# adding months column
final_df['month'] = final_df['started_at'].dt.month_name()
# final_df.head()
print('Unique month' ,final_df['month'].unique())

final_df.head()

Exploring the dataset to learn more about the distribution of the data.


In [ ]:
final_df.loc[:,['total_seconds']] = final_df.loc[:,['total_seconds']].astype('int64')

final_df.info()
final_df.head()

Storing the cleaned and merged data into a CSV to carryout further analysis in 'R'

In [ ]:
final_df.to_csv('/Users/locco/Desktop/Google_Analytics_Course/Case_Studies/Bike_share_Data/merged_csv.csv' ,index=False)